In [ ]:
import pickle
import numpy as np
import pandas as pd  
import matplotlib.pyplot as plt  
import seaborn as sns  
import scipy.stats as ss  
import pingouin as pg
from statsmodels.stats.multicomp import MultiComparison
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf 
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBRegressor, plot_importance
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.tsa.statespace.sarimax import SARIMAX
from scipy.stats import randint, uniform
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.linear_model import ridge_regression, Lasso, Ridge
import random
from functions import * 
import locale
from tabulate import tabulate
from sklearn.model_selection import GridSearchCV



# Inter
from statsmodels.formula.api import ols
import shap
import lime 
import lime.lime_tabular

# SARIMAX
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_pacf


# Options d'affichage.

# Configuration du style de visualisation des graphiques avec un style personnel (pitayasmoothie-dark).
#plt.style.use("https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle")

In [ ]:
# Import des données
df_reg = pd.read_csv('df_reg.csv')
df_valid_reg = pd.read_csv('df_valid_reg.csv')
df_valid = pd.read_csv('df_valid.csv')

In [ ]:

# Chargement des modèles
with open('model_lr.pkl', 'rb') as file:
    model_lr = pickle.load(file)
with open('model_rf.pkl', 'rb') as file:
    model_rf = pickle.load(file)
#with open('best_xgb_model.pkl', 'rb') as file:
#  best_xgb_model = pickle.load(file)
with open('model_ridge.pkl', 'rb') as file:
    model_ridge = pickle.load(file)
with open('model_lasso.pkl', 'rb') as file:
    model_lasso = pickle.load(file)
with open('model_dt.pkl', 'rb') as file:
    model_dt = pickle.load(file)


In [ ]:
# Division des données en ensemble d'entraînement et de test
X = df_reg.drop(columns=['Prix'])  # Variables indépendantes
y = df_reg['Prix']  # Variable dépendante
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

In [ ]:
# Colonnes à exclure
columns_to_exclude = [
    'Elasticite', 'ChocDemande', 'FacteurRarete', 'Categorie_Livres',
    'Categorie_Vetements','elasticite_cat_Inelastique', 'elasticite_cat_Neutre',
    'dayofweek', 'launch_dayofweek', 'launch_is_weekend', 'TempsDepuisLancement', 'is_weekend',
]

# Filtrer les données en excluant les colonnes définies
df_reg2 = df_reg.drop(columns=columns_to_exclude)

# Vérifier les colonnes restantes
print(df_reg2.columns)
# Préparation des données pour la régression
X = df_reg2.drop(columns=['Prix'])  # Variables indépendantes
y = df_reg2['Prix']  # Variable dépendante

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

In [ ]:
# Sélectionner une observation aléatoire
random_index = random.randint(0, len(df_valid) - 1)
random_observation = df_valid.iloc[random_index]
random_sku = random_observation['SKU']
random_date = random_observation['Date']

# Définir la fenêtre pour la moyenne mobile
window_size = 3
df_valid['Moving_Avg'] = df_valid['Prix'].rolling(window=window_size).mean()

# Filtrer le DataFrame de validation pour obtenir les données correspondantes au SKU et à la date
df_random = df_valid[(df_valid['SKU'] == random_sku) & (df_valid['Date'] == random_date)]

# Vérifier que l'observation est unique
if len(df_random) != 1:
    raise ValueError("L'observation sélectionnée n'est pas unique.")

# Stocker le vrai prix
y_actual = df_random['Prix'].values[0]
df_random_reg = df_valid_reg.loc[df_random.index]

# Comparer les colonnes de df_valid_reg et X_train pour trouver les colonnes manquantes
missing_cols = set(X_train.columns) - set(df_random_reg.columns)

# Ajouter les colonnes manquantes avec des zéros
for col in missing_cols:
    df_random_reg[col] = 0

# Assurer que l'ordre des colonnes est le même que dans X_train
df_random_reg = df_random_reg[X_train.columns]

# Préparer les données pour le modèle (sans le vrai prix)
X_random = df_random_reg

# Prédictions avec les différents modèles
y_pred_lr_log = model_lr.predict(X_random).tolist()[0]
y_pred_rf_log = model_rf.predict(X_random).tolist()[0]
# y_pred_xgb_log = best_xgb_model.predict(X_random).tolist()[0]

y_pred_ridge_log = model_ridge.predict(X_random).tolist()[0]
y_pred_lasso_log = model_lasso.predict(X_random).tolist()[0]
y_pred_dt_log = model_dt.predict(X_random).tolist()[0]

y_pred_moving_avg = df_random['Moving_Avg'].values[0]

# Inverser la transformation logarithmique pour obtenir les valeurs originales
y_pred_lr = np.exp(y_pred_lr_log)
y_pred_rf = np.exp(y_pred_rf_log)
#y_pred_xgb = np.exp(y_pred_xgb_log)
y_pred_ridge = np.exp(y_pred_ridge_log)
y_pred_lasso = np.exp(y_pred_lasso_log)
y_pred_dt = np.exp(y_pred_dt_log)

# Comparer les prédictions avec le vrai prix
print(f"SKU sélectionné: {random_sku}")
print(f"Date sélectionnée: {random_date}")
print(f"Prix réel: {y_actual}")
print(f"Prédiction Régression Linéaire (après transformation): {y_pred_lr}")
print(f"Prédiction Random Forest (après transformation): {y_pred_rf}")
#print(f"Prédiction XGBoost (après transformation): {y_pred_xgb}")
print(f"Prédiction Moyenne Mobile: {y_pred_moving_avg}")
print(f"Prédiction Ridge (après transformation): {y_pred_ridge}")
print(f"Prédiction Lasso (après transformation): {y_pred_lasso}")
print(f"Prédiction Decision Tree (après transformation): {y_pred_dt}")


In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np

categories = ['Electronique', 'Vetements', 'Livres']
category_random_skus = {}

# Sélectionner une SKU aléatoire par catégorie
for category in categories:
    df_category = df_valid[df_valid['Categorie'] == category]
    random_sku = df_category['SKU'].sample(n=1).values[0]
    category_random_skus[category] = random_sku

predictions = {}

for category, random_sku in category_random_skus.items():
    # Filtrer les données pour la SKU sélectionnée
    df_category_sku = df_valid[df_valid['SKU'] == random_sku].copy()
    df_category_sku['Moving_Avg'] = df_category_sku['Prix'].rolling(window=3).mean()
    
    # Correspondance dans df_valid_reg
    df_random_reg = df_valid_reg[df_valid['SKU'] == random_sku].copy()  # Utiliser .copy() pour éviter les modifications sur la vue

    # Ajouter des colonnes manquantes avec des zéros
    missing_cols = set(X_train.columns) - set(df_random_reg.columns)
    for col in missing_cols:
        df_random_reg[col] = 0

    # S'assurer que l'ordre des colonnes correspond à celui de X_train
    df_random_reg = df_random_reg[X_train.columns]

    # Prédictions
    X_random = df_random_reg
    y_actual = df_category_sku['Prix'].values
    y_pred_lr_log = model_lr.predict(X_random)
    y_pred_rf_log = model_rf.predict(X_random)
    #y_pred_xgb_log = best_xgb_model.predict(X_random)
    y_pred_ridge_log = model_ridge.predict(X_random)
    y_pred_lasso_log = model_lasso.predict(X_random)
    y_pred_dt_log = model_dt.predict(X_random)
    y_pred_moving_avg = df_category_sku['Moving_Avg'].values

    # Inverser la transformation logarithmique si applicable
    y_pred_lr = np.exp(y_pred_lr_log)
    y_pred_rf = np.exp(y_pred_rf_log)
    #y_pred_xgb = np.exp(y_pred_xgb_log)
    y_pred_ridge = np.exp(y_pred_ridge_log)
    y_pred_lasso = np.exp(y_pred_lasso_log)
    y_pred_dt = np.exp(y_pred_dt_log)
    
    predictions[category] = {
        'y_actual': y_actual,
        'y_pred_lr': y_pred_lr,
        'y_pred_rf': y_pred_rf,
        #'y_pred_xgb': y_pred_xgb,
        'y_pred_ridge': y_pred_ridge,
        'y_pred_lasso': y_pred_lasso,
        'y_pred_dt': y_pred_dt,
        'y_pred_moving_avg': y_pred_moving_avg,
        'dates': df_category_sku['Date'].values
    }

# Générer les graphiques
for category, preds in predictions.items():
    plt.figure(figsize=(14, 7))
    plt.plot(preds['dates'], preds['y_actual'], label='Prix réel', color='black')
    plt.plot(preds['dates'], preds['y_pred_lr'], label='Régression Linéaire', linestyle='--')
    plt.plot(preds['dates'], preds['y_pred_rf'], label='Random Forest', linestyle='--')
    #plt.plot(preds['dates'], preds['y_pred_xgb'], label='XGBoost', linestyle='--')
    plt.plot(preds['dates'], preds['y_pred_ridge'], label='Ridge', linestyle='--')
    plt.plot(preds['dates'], preds['y_pred_lasso'], label='Lasso', linestyle='--')
    plt.plot(preds['dates'], preds['y_pred_dt'], label='Decision Tree', linestyle='--')
    plt.plot(preds['dates'], preds['y_pred_moving_avg'], label='Moyenne Mobile', color='red')
    
    plt.xlabel('Date')
    plt.ylabel('Prix')
    plt.title(f'Évolution du prix pour {random_sku}')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()